# Experiments with Sequantial Chaining and Memory

In [1]:
import os
from langchain.chains import LLMChain
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import SimpleSequentialChain

In [2]:
os.environ['OPENAI_API_KEY'] = ''

In [3]:
llm = OpenAI(temperature=0.1)

In [4]:
def read_file(filename):
    with open(filename, 'r') as file:
        content = file.read()
    return content

def file_to_code(code_file):
    if code_file:
        code = code_file.read().decode('utf-8')
        return code

def add_tests(tests,file):
    with open(file, "a") as file_object:
        file_object.write(tests)

# Defining the template for the first chain

In [5]:
template = "You are an AI programming assistant." \
           "You dont respond to non-programming requests." \
           "Follow the user's requirements carefully & to the letter." \
           "if user asks for code, you must provide efficient and testing code." \
           "All methods in code should have export when initialised." \
           "The code should not have any formatting or syntax highlighting." \
           "Do not wrap the code in a code block." \
           "The code will be executed in Deno Runtime." \
           "Here is the coding request:\n" \
           "{request}"

In [6]:
prompt_template = PromptTemplate(input_variables=["request"], template=template)

In [7]:
code_chain = LLMChain(llm=llm, prompt=prompt_template)

# Defining the chain for test generation

In [14]:
template_test = "You are system which generates Deno.tests" \
           "You only generate tests for the provided code." \
           "You must include all essential imports such as:\n" \
           "assertion module\n" \
           "import method names ../code/temp.js;" \
           "Provided tests should not have any formatting or syntax highlighting." \
           "Do not wrap the tests in a code block." \
           "Here is the code:\n" \
           "{code}"

In [15]:
prompt_template = PromptTemplate(input_variables=["code"], template=template_test)

In [16]:
test_chain = LLMChain(llm=llm, prompt=prompt_template)

# Combining two chains

In [17]:
overall_chain = SimpleSequentialChain(
    chains=[code_chain, test_chain],
    verbose=True,
)

In [20]:
output = overall_chain.run("concise binary search")



> Entering new SimpleSequentialChain chain...


// Binary Search Algorithm
function binarySearch(arr, target) {
  let start = 0;
  let end = arr.length - 1;
  let middle = Math.floor((start + end) / 2);

  while (arr[middle] !== target && start <= end) {
    if (target < arr[middle]) {
      end = middle - 1;
    } else {
      start = middle + 1;
    }
    middle = Math.floor((start + end) / 2);
  }

  return arr[middle] === target ? middle : -1;
}

export { binarySearch };


// Tests
import { assertEquals } from "https://deno.land/std/testing/asserts.ts";
import { binarySearch } from "../code/temp.js";

Deno.test("binarySearch() returns the correct index", () => {
  const arr = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10];
  const target = 5;
  const expected = 4;
  const result = binarySearch(arr, target);
  assertEquals(result, expected);
});

Deno.test("binarySearch() returns -1 when the target is not found", () => {
  const arr = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10];
  const target = 11;
  const

In [27]:
print(output)



// Tests
import { assertEquals } from "https://deno.land/std/testing/asserts.ts";
import { binarySearch } from "../code/temp.js";

Deno.test("binarySearch() returns the correct index", () => {
  const arr = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10];
  const target = 5;
  const expected = 4;
  const result = binarySearch(arr, target);
  assertEquals(result, expected);
});

Deno.test("binarySearch() returns -1 when the target is not found", () => {
  const arr = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10];
  const target = 11;
  const expected = -1;
  const result = binarySearch(arr, target);
  assertEquals(result, expected);
});
